<a href="https://colab.research.google.com/github/dirun11/NLP_LAB/blob/main/MergingDatasets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re
import string
import nltk
import matplotlib.pyplot as plt
import seaborn as sns

from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import TweetTokenizer

from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA
nltk.download('punkt')

import warnings
warnings.filterwarnings("ignore")
#Run code from block 18(56) data cleaning steps from block 3 to 17

In [ ]:
data = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/reddit_wsb.csv")
data.head()

,title,score,id,url,comms_num,created,body,timestamp
0,"It's not about the money, it's about sending a...",55,l6ulcx,https://v.redd.it/6j75regs72e61,6,1.611863e+09,NaN,2021-01-28 21:37:41
1,Math Professor Scott Steiner says the numbers ...,110,l6uibd,https://v.redd.it/ah50lyny62e61,23,1.611862e+09,NaN,2021-01-28 21:32:10
2,Exit the system,0,l6uhhn,https://www.reddit.com/r/wallstreetbets/commen...,47,1.611862e+09,The CEO of NASDAQ pushed to halt trading “to g...,2021-01-28 21:30:35
3,NEW SEC FILING FOR GME! CAN SOMEONE LESS RETAR...,29,l6ugk6,https://sec.report/Document/0001193125-21-019848/,74,1.611862e+09,NaN,2021-01-28 21:28:57
4,"Not to distract from GME, just thought our AMC...",71,l6ufgy,https://i.redd.it/4h2sukb662e61.jpg,156,1.611862e+09,NaN,2021-01-28 21:26:56


In [ ]:
title_data = data[['title','timestamp']].copy()

title_data = title_data.dropna()

#Lower-case all post
title_data.title = title_data.title.str.lower()

#Remove handlers
title_data.title = title_data.title.apply(lambda x:re.sub('@[^\s]+','',x))

# Remove URLS
title_data.title = title_data.title.apply(lambda x:re.sub(r"http\S+", "", x))

# Remove all the special characters
title_data.title = title_data.title.apply(lambda x:' '.join(re.findall(r'\w+', x)))

#remove all single characters
title_data.title = title_data.title.apply(lambda x:re.sub(r'\s+[a-zA-Z]\s+', '', x))

# Substituting multiple spaces with single space
title_data.title = title_data.title.apply(lambda x:re.sub(r'\s+', ' ', x, flags=re.I))

#Remove Time From Timestamp
title_data.timestamp = pd.to_datetime(title_data.timestamp).dt.date

In [ ]:
sid = SIA()

title_data['sentiments']           = title_data['title'].apply(lambda x: sid.polarity_scores(' '.join(re.findall(r'\w+',x.lower()))))
title_data['Positive']   = title_data['sentiments'].apply(lambda x: x['pos']+1*(10**-6)) 
title_data['Neutral']    = title_data['sentiments'].apply(lambda x: x['neu']+1*(10**-6))
title_data['Negative']   = title_data['sentiments'].apply(lambda x: x['neg']+1*(10**-6))

title_data.drop(columns=['sentiments'],inplace=True)
title_data.head()

,title,timestamp,Positive,Neutral,Negative
0,itnot about the money itabout sendingmessage,2021-01-28,0.000001,1.000001,0.000001
1,math professor scott steiner says the numbers ...,2021-01-28,0.000001,0.728001,0.272001
2,exit the system,2021-01-28,0.000001,1.000001,0.000001
3,new sec filing for gme can someone less retard...,2021-01-28,0.138001,0.658001,0.204001
4,not to distract from gme just thought our amc ...,2021-01-28,0.119001,0.881001,0.000001


In [ ]:
data2 = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/pbl_dataset.csv")
data2.head()

,neg,neu,pos,compound,title,label,timestamp
0,0.000,0.684,0.316,0.5707,A little hopium goes a long way! $11k bullish ...,1,2022-10-13
1,0.368,0.632,0.000,-0.5423,If bad cpi = homeless for me. 🥹😀🪦🪦💀💀,-1,2022-10-13
2,0.000,1.000,0.000,0.0000,Put holders today,0,2022-10-13
3,0.000,0.844,0.156,0.5719,All red today - but the Hyper Efficient Market...,1,2022-10-13
4,0.000,1.000,0.000,0.0000,"Y'all called it one day too early, PUT options...",0,2022-10-13


In [ ]:
data2 = data2.drop(['compound','label','neg','neu','pos'], axis=1)

In [ ]:
data2.head()

,title,timestamp
0,A little hopium goes a long way! $11k bullish ...,2022-10-13
1,If bad cpi = homeless for me. 🥹😀🪦🪦💀💀,2022-10-13
2,Put holders today,2022-10-13
3,All red today - but the Hyper Efficient Market...,2022-10-13
4,"Y'all called it one day too early, PUT options...",2022-10-13


In [ ]:
data2 = data2.dropna()

#Lower-case all post
data2.title = data2.title.str.lower()

#Remove handlers
data2.title = data2.title.apply(lambda x:re.sub('@[^\s]+','',x))

# Remove URLS
data2.title = data2.title.apply(lambda x:re.sub(r"http\S+", "", x))

# Remove all the special characters
data2.title = data2.title.apply(lambda x:' '.join(re.findall(r'\w+', x)))

#remove all single characters
data2.title = data2.title.apply(lambda x:re.sub(r'\s+[a-zA-Z]\s+', '', x))

# Substituting multiple spaces with single space
data2.title = data2.title.apply(lambda x:re.sub(r'\s+', ' ', x, flags=re.I))

In [ ]:
data2.head()

,title,timestamp
0,a little hopium goeslong way 11k bullish gain ...,2022-10-13
1,if bad cpi homeless for me,2022-10-13
2,put holders today,2022-10-13
3,all red today but the hyper efficient market h...,2022-10-13
4,y all called it one day too early put options ...,2022-10-13


In [ ]:
sid = SIA()

data2['sentiments']           = data2['title'].apply(lambda x: sid.polarity_scores(' '.join(re.findall(r'\w+',x.lower()))))
data2['Positive']   = data2['sentiments'].apply(lambda x: x['pos']+1*(10**-6)) 
data2['Neutral']    = data2['sentiments'].apply(lambda x: x['neu']+1*(10**-6))
data2['Negative']   = data2['sentiments'].apply(lambda x: x['neg']+1*(10**-6))

data2.drop(columns=['sentiments'],inplace=True)
data2.head()

,title,timestamp,Positive,Neutral,Negative
0,a little hopium goeslong way 11k bullish gain ...,2022-10-13,0.327001,0.673001,0.000001
1,if bad cpi homeless for me,2022-10-13,0.000001,0.588001,0.412001
2,put holders today,2022-10-13,0.000001,1.000001,0.000001
3,all red today but the hyper efficient market h...,2022-10-13,0.156001,0.844001,0.000001
4,y all called it one day too early put options ...,2022-10-13,0.000001,1.000001,0.000001


In [ ]:
title_data = title_data.append(data2)

In [ ]:
title_data.head(60000)

,title,timestamp,Positive,Neutral,Negative
0,itnot about the money itabout sendingmessage,2021-01-28,0.000001,1.000001,0.000001
1,math professor scott steiner says the numbers ...,2021-01-28,0.000001,0.728001,0.272001
2,exit the system,2021-01-28,0.000001,1.000001,0.000001
3,new sec filing for gme can someone less retard...,2021-01-28,0.138001,0.658001,0.204001
4,not to distract from gme just thought our amc ...,2021-01-28,0.119001,0.881001,0.000001
...,...,...,...,...,...
2479,go kathy,2022-10-12,0.000001,1.000001,0.000001
2480,a little hopium goeslong way 11k bullish gain ...,2022-10-12,0.327001,0.673001,0.000001
2481,at firstthought this was one of yall,2022-10-12,0.000001,1.000001,0.000001
2482,what happenedpow you used to be cool,2022-10-12,0.277001,0.723001,0.000001


In [ ]:
data3 = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/pbl_dataset(final).csv")
data3.head(100000)

,title,timestamp,Positive,Neutral,Negative
0,itnot about the money itabout sendingmessage,2021-01-28,0.000001,1.000001,0.000001
1,math professor scott steiner says the numbers ...,2021-01-28,0.000001,0.728001,0.272001
2,exit the system,2021-01-28,0.000001,1.000001,0.000001
3,new sec filing for gme can someone less retard...,2021-01-28,0.138001,0.658001,0.204001
4,not to distract from gme just thought our amc ...,2021-01-28,0.119001,0.881001,0.000001
...,...,...,...,...,...
55666,go kathy,2022-10-12,0.000001,1.000001,0.000001
55667,a little hopium goeslong way 11k bullish gain ...,2022-10-12,0.327001,0.673001,0.000001
55668,at firstthought this was one of yall,2022-10-12,0.000001,1.000001,0.000001
55669,what happenedpow you used to be cool,2022-10-12,0.277001,0.723001,0.000001


In [ ]:
data4 = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/GMEData(timestamps).csv")
data4.head(1000)

,High,Low,Open,Close,Volume,Adj Close,timestamp
0,1.617500,1.517500,1.535000,1.577500,17814400.0,1.577500,2020-01-02
1,1.562500,1.460000,1.552500,1.470000,14175600.0,1.470000,2020-01-03
2,1.477500,1.400000,1.450000,1.462500,13579200.0,1.462500,2020-01-06
3,1.457500,1.360000,1.442500,1.380000,20912000.0,1.380000,2020-01-07
4,1.462500,1.352500,1.372500,1.430000,22517600.0,1.430000,2020-01-08
...,...,...,...,...,...,...,...
699,26.299999,23.950001,24.870001,25.270000,4195900.0,25.270000,2022-10-11
700,26.889999,25.270000,25.959999,25.360001,3284400.0,25.360001,2022-10-12
701,26.590000,23.910000,24.420000,25.559999,3516200.0,25.559999,2022-10-13
702,26.370001,24.629999,25.770000,24.629999,2773500.0,24.629999,2022-10-14
